<a href="https://colab.research.google.com/github/isabelacfs/MVP_Data_Engineering/blob/main/faker_mvp_engenharia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso: Pós-graduação Lato Sensu em Data Science & Analytics (PUC-Rio)

Aluna: Isabela Costa Ferreira da Silva

Matrícula:

Este pedaço de código foi utilizado para mascarar os dados dos usuários e dados referentes ao negócio que administramos de forma que pudéssemos demonstrar o processo de construção da base de informações para o trabalho de conclusão (MVP) da disciplina de Engenharia de dados da Puc-Rio.

In [11]:
pip install faker # instalando a biblioteca que irá nos ajudar a mascarar os dados

In [12]:
# importando pacotes necessários
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [13]:


# =============================================
# Configuração para reprodutibilidade e país Brasil
# =============================================
fake = Faker('pt_BR')
Faker.seed(42)
np.random.seed(42)

# =============================================
# 1. Ler os arquivos CSV originais
# =============================================
cadastro = pd.read_csv("cadastros-mvp.csv")   # SUBSTITUA PELOS SEUS ARQUIVOS
consumo  = pd.read_csv("consumo-mvp.csv")    # SUBSTITUA PELOS SEUS ARQUIVOS

# =============================================
# 2. Criar novo ID sequencial bonito (100001, 100002...)
# =============================================
# Ordena para garantir que o mapeamento seja sempre o mesmo
cadastro = cadastro.sort_values(by='id').reset_index(drop=True)
cadastro['id_cliente_novo'] = range(100001, 100001 + len(cadastro))

# =============================================
# 3. Substituir nome de usuário por nome falso realista
# =============================================
cadastro['nome_falso'] = [fake.name() for _ in range(len(cadastro))]

# =============================================
# 4. Criar numeros de telefone falsos
# =============================================
cadastro['telefone_falso'] = [fake.phone_number() for _ in range(len(cadastro))]

# =============================================
# 5. Criar tabela cadastro pública (sem CPF, sem nome real)
# =============================================
cadastro_publico = cadastro[[
    'id_cliente_novo',
    'nome_falso',
    'telefone_falso',
    'dt_nascimento',
    'dt_registro'
]].copy()

cadastro_publico = cadastro_publico.rename(columns={
    'id_cliente_novo': 'id_cliente',
    'nome_falso':     'nome_cliente',
    'telefone_falso': 'telefone'
})

# Reordenar colunas
cadastro_publico = cadastro_publico[[
    'id_cliente', 'nome_cliente','telefone', 'dt_nascimento', 'dt_registro'
]]

# =============================================
# 6. Atualizar tabela consumo com o novo ID
# =============================================
# Dicionário: CPF antigo → novo id inteiro
mapeamento_cpf_para_novo = dict(zip(cadastro['id'], cadastro['id_cliente_novo']))

consumo_publico = consumo.copy()
consumo_publico['id_cliente'] = consumo_publico['id'].map(mapeamento_cpf_para_novo)

# Remove a coluna antiga que tinha o CPF
consumo_publico = consumo_publico.drop(columns=['id'])

# Reordenar colunas
consumo_publico = consumo_publico[[
    'id_cliente', 'qtde_compras', 'valor_compras', 'periodo_consumo'
]]

# =============================================
# 7. Ofuscar valor_compras (recomendado para total segurança)
# =============================================
# Opção 1: adicionar ruído aleatório ±30%
consumo_publico['valor_compras'] = (
    consumo_publico['valor_compras'] * np.random.uniform(0.7, 1.3, len(consumo_publico))
).round(2)

# =============================================
# 8. transformar em inteiro o id do cliente
consumo_publico['id_cliente'] = consumo_publico['id_cliente'].astype(int)
# =============================================

# =============================================
# 9. Mostrar resultados
# =============================================

print("Anonimização concluída com sucesso!")
print(f"Clientes processados: {len(cadastro_publico):,}")
print(f"Transações mantidas:   {len(consumo_publico):,}")
print("\nPrimeiras linhas do cadastro público:")
print(cadastro_publico.head())
print("\nPrimeiras linhas do consumo público:")
print(consumo_publico.head())

Anonimização concluída com sucesso!
Clientes processados: 2,705
Transações mantidas:   4,587

Primeiras linhas do cadastro público:
   id_cliente          nome_cliente             telefone dt_nascimento  \
0      100001          Brenda Alves      (021) 9083-5723    1993-05-18   
1      100002  Sra. Isabelly Câmara  +55 (021) 4665-1364    1999-02-17   
2      100003            Cauã Rocha      (011) 2493 6312    1981-03-10   
3      100004    Dra. Aurora Pastor  +55 (071) 0347-7866    1954-05-28   
4      100005     Ana Beatriz Alves  +55 (051) 3242 0448    1983-09-01   

           dt_registro  
0  2024-07-25 10:44:15  
1  2025-01-12 11:58:23  
2  2024-03-14 16:31:15  
3  2024-04-21 11:09:12  
4  2024-02-29 21:29:29  

Primeiras linhas do consumo público:
   id_cliente  qtde_compras  valor_compras periodo_consumo
0    102337.0            13         203.16         2024-04
1    100058.0            12         257.64         2024-04
2    102246.0            12         231.03         2024-04